In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from pathlib import Path
import sys

import importlib


# allow imports from src/
PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

# import downloader functions
from src.model.regression_utils import train_model

In [5]:
# Load all data
df_all = pd.read_csv("../data/processed/df_all.csv")
df_all['iso_timestamp'] = pd.to_datetime(df_all['iso_timestamp'], utc=True, format='mixed')

# Change columns to numerics
numeric_cols = [
    "channels_in", "channels_out", "channels_unknown", "channels_all",
    "site_temperature", "site_rain_accumulation", "site_snow_accumulation"]
df_all[numeric_cols] = df_all[numeric_cols].apply(pd.to_numeric, errors='coerce')

df_all["count"] = df_all[["channels_out", "channels_in", "channels_unknown"]].fillna(0).sum(axis=1)

print(f"Loaded {len(df_all):,} total records")
print(f"Cities: {df_all['domain_name'].nunique()}")
print(f"\nAvailable cities:")
for city in sorted(df_all['domain_name'].unique()):
    n_stations = df_all[df_all['domain_name'] == city]['counter_site'].nunique()
    print(f"  {city:40} ({n_stations:2} stations)")

Loaded 6,080,705 total records
Cities: 22

Available cities:
  Landeshauptstadt Stuttgart               (15 stations)
  Landkreis Böblingen                      ( 2 stations)
  Landratsamt Ostalbkreis                  (11 stations)
  Landratsamt Rems-Murr-Kreis              (13 stations)
  Ravensburg Tws Gmbh & Co. Kg             ( 7 stations)
  Regierungspräsidium Stuttgart Aussenstelle Heilbronn ( 1 stations)
  Stadt Bad Säckingen                      ( 2 stations)
  Stadt Freiburg                           ( 3 stations)
  Stadt Heidelberg                         (15 stations)
  Stadt Heilbronn                          ( 4 stations)
  Stadt Karlsruhe                          ( 1 stations)
  Stadt Kirchheim Unter Teck               ( 1 stations)
  Stadt Konstanz                           ( 6 stations)
  Stadt Ludwigsburg                        (15 stations)
  Stadt Lörrach                            ( 2 stations)
  Stadt Mannheim                           (14 stations)
  Stadt Offenbu

In [6]:
# Load feature data
df_features = pd.read_csv('df_features.csv')
df_features['iso_timestamp'] = pd.to_datetime(df_features['iso_timestamp'], utc=True)

print(f"Loaded {len(df_features):,} feature records")

FileNotFoundError: [Errno 2] No such file or directory: 'df_features.csv'

In [4]:
# Define cities to process (you can modify this list)
cities_to_process = [
    "Stadt Freiburg",
    "Stadt Heidelberg",
    "Landeshauptstadt Stuttgart",
    "Stadt Mannheim",
    "Stadt Reutlingen",
    "Stadt Konstanz",
    "Stadt Tübingen",
    "Stadt Ludwigsburg",
    "Ravensburg Tws Gmbh & Co. Kg"
]

# Outlier threshold
OUTLIER_THRESHOLD = 1000

print(f"Will process {len(cities_to_process)} cities")
print(f"Outlier threshold: >{OUTLIER_THRESHOLD} hourly counts")

Will process 9 cities
Outlier threshold: >1000 hourly counts


In [7]:
# Function to train model with optional outlier removal
def train_with_outlier_option(df_station, station_name, df_city, remove_outliers=False, threshold=500):
    """
    Train model with option to remove outliers
    
    Returns: results dict with additional 'n_train_samples' field, or None if failed
    """
    if remove_outliers:
        # Filter out outliers from station data
        df_station = df_station[df_station['channels_all'] <= threshold].copy()
        # Also filter city data to maintain consistency
        df_city = df_city[df_city['channels_all'] <= threshold].copy()
    
    if len(df_station) < 1000:
        return None
    
    results, model, features = train_model(df_station, station_name, df_city)
    
    if results is None:
        return None
    
    # Add training sample count
    # train_model uses 80/20 split
    # We need to replicate the same filtering logic to get accurate count
    from regression_utils import add_station_features
    
    # Get all stations data for pivot
    df_pivot_all = df_city.pivot_table(
        index='iso_timestamp',
        columns='counter_site',
        values='channels_all'
    )
    valid_timestamps = df_pivot_all.dropna().index
    df_station_filtered = df_station[df_station['iso_timestamp'].isin(valid_timestamps)].copy()
    
    # Check if we have any data after filtering
    if len(df_station_filtered) == 0:
        return None
    
    df_model = add_station_features(df_station_filtered, station_name, df_city)
    
    # Check if we have enough data for training
    if len(df_model) == 0:
        return None
    
    split_idx = int(len(df_model) * 0.8)
    
    # Skip if no training samples
    if split_idx == 0:
        return None
    
    results['n_train_samples'] = split_idx
    
    return results

In [13]:
################################################
# PROCESS ALL CITIES                           #
################################################

all_city_results = {}

for city_name in cities_to_process:
    print(f"\n{'='*80}")
    print(f"PROCESSING: {city_name}")
    print(f"{'='*80}")
    
    # Get city data
    df_city = df_features[df_features['domain_name'] == city_name].copy()
    
    if len(df_city) == 0:
        print(f"⊗ No data found for {city_name}")
        continue
    
    stations = df_city['counter_site'].unique()
    print(f"\nStations: {len(stations)}")
    print(f"Records: {len(df_city):,}")
    
    city_results = []
    
    for station_idx, station in enumerate(stations, 1):
        print(f"\n[{station_idx}/{len(stations)}] {station[:50]}")
        
        # Get station data
        df_station = df_city[df_city['counter_site'] == station].copy()
        df_station = df_station.dropna(subset=['channels_all'])
        df_station = df_station.sort_values('iso_timestamp')
        
        if len(df_station) < 1000:
            print(f"    ⊗ Skipped - only {len(df_station)} records")
            continue
        
        # Train WITHOUT outlier removal
        print("    Training WITH outliers...")
        results_with = train_with_outlier_option(df_station, station, df_city, remove_outliers=False)
        
        if results_with is None:
            print("    ⊗ Failed to train model with outliers")
            continue
        
        # Train WITH outlier removal
        print("    Training WITHOUT outliers (threshold >500)...")
        results_without = train_with_outlier_option(df_station, station, df_city, 
                                                      remove_outliers=True, threshold=OUTLIER_THRESHOLD)
        
        if results_without is None:
            print("    ⊗ Failed to train model without outliers")
            continue
        
        # Combine results
        combined = {
            'city': city_name,
            'station': station,
            'n_features': results_with['n_features'],
            'n_train_samples': results_with['n_train_samples'],
            'n_train_samples_no_outliers': results_without['n_train_samples'],
            # With outliers
            'test_r2_with': results_with['test_r2'],
            'test_rmse_with': results_with['test_rmse'],
            'test_mae_with': results_with['test_mae'],
            # Without outliers
            'test_r2_without': results_without['test_r2'],
            'test_rmse_without': results_without['test_rmse'],
            'test_mae_without': results_without['test_mae'],
            # Difference
            'delta_r2': results_without['test_r2'] - results_with['test_r2']
        }
        
        city_results.append(combined)
        
        print(f"    WITH outliers:    R²={results_with['test_r2']:.4f} | RMSE={results_with['test_rmse']:.2f} | n={results_with['n_train_samples']}")
        print(f"    WITHOUT outliers: R²={results_without['test_r2']:.4f} | RMSE={results_without['test_rmse']:.2f} | n={results_without['n_train_samples']}")
        print(f"    ΔR² = {combined['delta_r2']:+.4f}")
    
    all_city_results[city_name] = city_results
    
    # Print city summary
    if len(city_results) > 0:
        df_city_results = pd.DataFrame(city_results)
        print(f"\n{'-'*80}")
        print(f"SUMMARY - {city_name}")
        print(f"{'-'*80}")
        print(f"Successfully trained models for {len(city_results)} stations")
        print(f"\nAverage R² WITH outliers:    {df_city_results['test_r2_with'].mean():.4f} ± {df_city_results['test_r2_with'].std():.4f}")
        print(f"Average R² WITHOUT outliers: {df_city_results['test_r2_without'].mean():.4f} ± {df_city_results['test_r2_without'].std():.4f}")
        print(f"Average ΔR²:                 {df_city_results['delta_r2'].mean():+.4f} ± {df_city_results['delta_r2'].std():.4f}")

print(f"\n\n{'='*80}")
print("ALL CITIES PROCESSED")
print(f"{'='*80}")


PROCESSING: Stadt Freiburg

Stations: 10
Records: 481,718

[1/10] Wiwilibrücke
    Training WITH outliers...

Stations: 10
Records: 481,718

[1/10] Wiwilibrücke
    Training WITH outliers...
    Complete coverage: 6,067/112,321 hours (5.4%)
    Complete coverage: 6,067/112,321 hours (5.4%)
    Time frame: 2024-05-22 22:00:00+00:00 to 2025-10-31 01:00:00+00:00 (526 days, 6,067 hours)
    Time frame: 2024-05-22 22:00:00+00:00 to 2025-10-31 01:00:00+00:00 (526 days, 6,067 hours)
    Training WITHOUT outliers (threshold >500)...
    Training WITHOUT outliers (threshold >500)...
    Complete coverage: 5,721/112,270 hours (5.1%)
    Complete coverage: 5,721/112,270 hours (5.1%)
    Time frame: 2024-05-22 22:00:00+00:00 to 2025-10-31 01:00:00+00:00 (526 days, 5,721 hours)
    Time frame: 2024-05-22 22:00:00+00:00 to 2025-10-31 01:00:00+00:00 (526 days, 5,721 hours)
    WITH outliers:    R²=0.9733 | RMSE=53.05 | n=4853
    WITHOUT outliers: R²=0.9712 | RMSE=50.37 | n=4576
    ΔR² = -0.0021

[

In [14]:
################################################
# DISPLAY RESULTS PER CITY                     #
################################################

for city_name, city_results in all_city_results.items():
    if len(city_results) == 0:
        continue
    
    df_city_results = pd.DataFrame(city_results)
    
    print(f"\n{'='*120}")
    print(f"{city_name} (outlier threshold: >{OUTLIER_THRESHOLD})")
    print(f"{'='*120}")
    print(f"\n| Station | R² (with) | RMSE (with) | MAE (with) | R² (no outl) | RMSE (no outl) | MAE (no outl) | ΔR² | Features | n_train | n_train_no_outl |")
    print(f"|---------|-----------|-------------|------------|--------------|----------------|---------------|-----|----------|---------|-----------------|")
    
    for _, row in df_city_results.iterrows():
        print(f"| {row['station'][:30]:30} | "
              f"{row['test_r2_with']:9.4f} | "
              f"{row['test_rmse_with']:11.2f} | "
              f"{row['test_mae_with']:10.2f} | "
              f"{row['test_r2_without']:12.4f} | "
              f"{row['test_rmse_without']:14.2f} | "
              f"{row['test_mae_without']:13.2f} | "
              f"{row['delta_r2']:+7.4f} | "
              f"{row['n_features']:8} | "
              f"{row['n_train_samples']:7} | "
              f"{row['n_train_samples_no_outliers']:15} |")
    
    print(f"\nSummary Statistics:")
    print(f"  Outlier threshold:         >{OUTLIER_THRESHOLD} hourly counts")
    print(f"  Avg R² (with outliers):    {df_city_results['test_r2_with'].mean():.4f} ± {df_city_results['test_r2_with'].std():.4f}")
    print(f"  Avg R² (no outliers):      {df_city_results['test_r2_without'].mean():.4f} ± {df_city_results['test_r2_without'].std():.4f}")
    print(f"  Avg ΔR²:                   {df_city_results['delta_r2'].mean():+.4f} ± {df_city_results['delta_r2'].std():.4f}")
    print(f"  Avg RMSE (with outliers):  {df_city_results['test_rmse_with'].mean():.2f} ± {df_city_results['test_rmse_with'].std():.2f}")
    print(f"  Avg RMSE (no outliers):    {df_city_results['test_rmse_without'].mean():.2f} ± {df_city_results['test_rmse_without'].std():.2f}")


Stadt Freiburg (outlier threshold: >1000)

| Station | R² (with) | RMSE (with) | MAE (with) | R² (no outl) | RMSE (no outl) | MAE (no outl) | ΔR² | Features | n_train | n_train_no_outl |
|---------|-----------|-------------|------------|--------------|----------------|---------------|-----|----------|---------|-----------------|
| Wiwilibrücke                   |    0.9733 |       53.05 |      38.02 |       0.9712 |          50.37 |         36.16 | -0.0021 |       12 |    4853 |            4576 |
| FR2 Güterbahn / Ferd.-Weiß-Str |    0.9682 |       25.60 |      16.49 |       0.9598 |          25.36 |         16.17 | -0.0083 |       12 |    4853 |            4576 |
| FR1 Dreisam / Otto-Wels-Str.   |    0.7984 |      109.33 |      69.04 |       0.7507 |         108.87 |         69.24 | -0.0478 |       12 |    4853 |            4576 |
| FR3 Eschholzstr. / Egonstr. ei |    0.9531 |       21.28 |      15.63 |       0.9443 |          21.01 |         15.33 | -0.0087 |       12 |    4853 |   

In [ ]:
################################################
# HEIDELBERG ONLY (excluding specific station, since otherwise we have no overlap) #
################################################

city_name = "Stadt Heidelberg"
exclude_station = "Ernst-Walz-Brücke West - alt"
# print station names
print(df_features[df_features['domain_name'] == 'Stadt Heidelberg']['counter_site'].unique())


print(f"\n{'='*80}")
print(f"PROCESSING: {city_name} (excluding {exclude_station})")
print(f"{'='*80}")

# Get city data
df_city = df_features[df_features['domain_name'] == city_name].copy()

if len(df_city) == 0:
    print(f"⊗ No data found for {city_name}")
else:
    # Exclude the specific station
    df_city = df_city[df_city['counter_site'] != exclude_station].copy()
    
    print("stations before exclusion:", len(stations))
    stations = df_city['counter_site'].unique()
    print(f"\nStations (after exclusion): {len(stations)}")
    print(f"Records: {len(df_city):,}")
    
    heidelberg_results = []
    
    for station_idx, station in enumerate(stations, 1):
        print(f"\n[{station_idx}/{len(stations)}] {station[:50]}")
        
        # Get station data
        df_station = df_city[df_city['counter_site'] == station].copy()
        df_station = df_station.dropna(subset=['channels_all'])
        df_station = df_station.sort_values('iso_timestamp')
        
        if len(df_station) < 1000:
            print(f"    ⊗ Skipped - only {len(df_station)} records")
            continue
        
        # Train WITHOUT outlier removal
        print("    Training WITH outliers...")
        results_with = train_with_outlier_option(df_station, station, df_city, remove_outliers=False)
        
        if results_with is None:
            print("    ⊗ Failed to train model with outliers")
            continue
        
        # Train WITH outlier removal
        print(f"    Training WITHOUT outliers (threshold >{OUTLIER_THRESHOLD})...")
        results_without = train_with_outlier_option(df_station, station, df_city, 
                                                      remove_outliers=True, threshold=OUTLIER_THRESHOLD)
        
        if results_without is None:
            print("    ⊗ Failed to train model without outliers")
            continue
        
        # Combine results
        combined = {
            'city': city_name,
            'station': station,
            'n_features': results_with['n_features'],
            'n_train_samples': results_with['n_train_samples'],
            'n_train_samples_no_outliers': results_without['n_train_samples'],
            # With outliers
            'test_r2_with': results_with['test_r2'],
            'test_rmse_with': results_with['test_rmse'],
            'test_mae_with': results_with['test_mae'],
            # Without outliers
            'test_r2_without': results_without['test_r2'],
            'test_rmse_without': results_without['test_rmse'],
            'test_mae_without': results_without['test_mae'],
            # Difference
            'delta_r2': results_without['test_r2'] - results_with['test_r2']
        }
        
        heidelberg_results.append(combined)
        
        print(f"    WITH outliers:    R²={results_with['test_r2']:.4f} | RMSE={results_with['test_rmse']:.2f} | n={results_with['n_train_samples']}")
        print(f"    WITHOUT outliers: R²={results_without['test_r2']:.4f} | RMSE={results_without['test_rmse']:.2f} | n={results_without['n_train_samples']}")
        print(f"    ΔR² = {combined['delta_r2']:+.4f}")
    
    # Display results
    if len(heidelberg_results) > 0:
        df_heidelberg = pd.DataFrame(heidelberg_results)
        
        print(f"\n{'='*120}")
        print(f"{city_name} (outlier threshold: >{OUTLIER_THRESHOLD}, excluding {exclude_station})")
        print(f"{'='*120}")
        print(f"\n| Station | R² (with) | RMSE (with) | MAE (with) | R² (no outl) | RMSE (no outl) | MAE (no outl) | ΔR² | Features | n_train | n_train_no_outl |")
        print(f"|---------|-----------|-------------|------------|--------------|----------------|---------------|-----|----------|---------|-----------------|")
        
        for _, row in df_heidelberg.iterrows():
            print(f"| {row['station'][:30]:30} | "
                  f"{row['test_r2_with']:9.4f} | "
                  f"{row['test_rmse_with']:11.2f} | "
                  f"{row['test_mae_with']:10.2f} | "
                  f"{row['test_r2_without']:12.4f} | "
                  f"{row['test_rmse_without']:14.2f} | "
                  f"{row['test_mae_without']:13.2f} | "
                  f"{row['delta_r2']:+7.4f} | "
                  f"{row['n_features']:8} | "
                  f"{row['n_train_samples']:7} | "
                  f"{row['n_train_samples_no_outliers']:15} |")
        
        print(f"\nSummary Statistics:")
        print(f"  Excluded station:          {exclude_station}")
        print(f"  Outlier threshold:         >{OUTLIER_THRESHOLD} hourly counts")
        print(f"  Avg R² (with outliers):    {df_heidelberg['test_r2_with'].mean():.4f} ± {df_heidelberg['test_r2_with'].std():.4f}")
        print(f"  Avg R² (no outliers):      {df_heidelberg['test_r2_without'].mean():.4f} ± {df_heidelberg['test_r2_without'].std():.4f}")
        print(f"  Avg ΔR²:                   {df_heidelberg['delta_r2'].mean():+.4f} ± {df_heidelberg['delta_r2'].std():.4f}")
        print(f"  Avg RMSE (with outliers):  {df_heidelberg['test_rmse_with'].mean():.2f} ± {df_heidelberg['test_rmse_with'].std():.2f}")
        print(f"  Avg RMSE (no outliers):    {df_heidelberg['test_rmse_without'].mean():.2f} ± {df_heidelberg['test_rmse_without'].std():.2f}")

['Plöck' 'Gaisbergstraße' 'Mannheimer Straße'
 'Ernst-Walz-Brücke Querschnitt' 'Ernst-Walz-Brücke West - alt'
 'Thedor-Heuss-Brücke Querschnitt' 'Rohrbacher Straße Querschnitt'
 'Liebermannstraße' 'Schlierbacher Landstraße' 'Ziegelhäuser Landstraße'
 'Kurfürstenanlage Querschnitt' 'Hardtstraße'
 'Berliner Straße Querschnitt' 'Eppelheimer Str. Querschnitt'
 'Bahnstadtpromenade']

PROCESSING: Stadt Heidelberg (excluding Ernst-Walz-Brücke West - alt)
stations before exclusion: 15

Stations (after exclusion): 14
Records: 820,368

[1/14] Plöck
    Training WITH outliers...
    Complete coverage: 11,555/99,956 hours (11.6%)
    Time frame: 2020-01-31 23:00:00+00:00 to 2022-05-10 05:00:00+00:00 (829 days, 11,555 hours)
    Training WITHOUT outliers (threshold >1000)...
    Complete coverage: 11,487/99,955 hours (11.5%)
    Time frame: 2020-01-31 23:00:00+00:00 to 2022-05-10 04:00:00+00:00 (829 days, 11,487 hours)
    WITH outliers:    R²=0.8339 | RMSE=51.94 | n=9244
    WITHOUT outliers: R²=0